In [ ]:
import os
import torch
from transformers import ( 
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    )
from pprint import PrettyPrinter
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import datasets
import psutil
import json

In [ ]:
def memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"RSS: {mem_info.rss / (1024 ** 2):.2f} MB")  # Resident Set Size
    print(f"VMS: {mem_info.vms / (1024 ** 2):.2f} MB")  # Virtual Memory Size
    print(f"VMS: {mem_info.vms / (1024 ** 2):.2f} MB")  # Virtual Memory Size

memory_usage()

In [3]:
from dotenv import load_dotenv
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

In [ ]:
path_to_parquet = "/data/ceph/storage/data-tmp/current/kavo1286/personal/data/ft_data/"
data_files = (file for file in os.listdir(path_to_parquet) if file.endswith('parquet'))
counter = 0
pd_dataset = None
for file in data_files:
    if counter == 0:
        pd_dataset = pd.read_parquet(path_to_parquet + file, engine='pyarrow')
    df = pd.read_parquet(path_to_parquet + file, engine='pyarrow')
    pd_dataset = pd.concat([pd_dataset, df])
    df = None
    counter += 1
    break
print(len(pd_dataset))
memory_usage()
pd_dataset.sample(frac=1, random_state=42)

In [ ]:
train_df = pd_dataset[['text']].iloc[:50000]
val_df = pd_dataset[['text']].iloc[50000:51000]
train_dataset = datasets.Dataset.from_pandas(train_df,preserve_index=False)
val_dataset = datasets.Dataset.from_pandas(val_df,preserve_index=False)
print(train_dataset)

In [6]:
lora_r = 16 #lora attention dimension/ rank
lora_alpha = 16 #lora scaling parameter
lora_dropout = 0.1 #lora dropout probability

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

#output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

#number of training epochs
num_train_epochs = 1

#enable fp16/bf16 training (set bf16 to True when using A100 GPU in google colab)
fp16 = False
bf16 = False

#batch size per GPU for training
per_device_train_batch_size = 4

#batch size per GPU for evaluation
per_device_eval_batch_size = 4

#gradient accumulation steps - No of update steps
gradient_accumulation_steps = 4

#learning rate
learning_rate = 2e-4

#weight decay
weight_decay = 0.001

#Gradient clipping(max gradient Normal)
max_grad_norm = 0.3

#optimizer to use
optim = "paged_adamw_32bit"

#learning rate scheduler
lr_scheduler_type = "cosine"

#seed for reproducibility
seed = 1

#Number of training steps
max_steps = -1

#Ratio of steps for linear warmup
warmup_ratio = 0.03

#group sequnces into batches with same length
group_by_length = False

#save checkpoint every X updates steps
save_steps = 500

#Log at every X updates steps
logging_steps = 100

packing = False

#load the entire model on the GPU
device_map = {"":0}

In [7]:
model_name = "mistralai/Mistral-7B-v0.3"
new_model = "Mistral-7B-sim-qlora"

In [ ]:
#load tokenizer and model with QLoRA config
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,)

#cheking GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("="*80)
        print("Your GPU supports bfloat16, you are getting accelerate training with bf16= True")
        print("="*80)

#load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = device_map,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code = True)

#Load QLoRA config
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r  = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [ ]:
# Function to compute tokenized lengths
def compute_token_length(example):
    tokenized_length = len(tokenizer(example["text"], truncation=False)["input_ids"])
    return {"length": tokenized_length}

# Compute tokenized lengths and add 'length' column to the dataset
train_dataset = train_dataset.map(compute_token_length)

# Get the maximum sequence length
max_seq_length = max(train_dataset["length"])
print(f"Maximum sequence length in train dataset: {max_seq_length}")
if max_seq_length <= 2048:
    max_seq_length = 2048
    

In [ ]:
#Set Training parameters
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    fp16 = fp16,
    bf16 = bf16,
    max_grad_norm = max_grad_norm,
    weight_decay = weight_decay,
    lr_scheduler_type = lr_scheduler_type,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    max_steps = max_steps,
    report_to = "tensorboard",
)

#SFT Trainer
trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_arguments,
    tokenizer = tokenizer,
    packing = packing,
)

#Start training
trainer.train()

In [ ]:
print(f"3: Used CUDA memory: {torch.cuda.memory_allocated() / 1e6} MB")

In [29]:
trainer.save_model('adapter')

In [ ]:
output_file_name = 'outputs/outputs_' + new_model + '.json'
with open(output_file_name, 'w') as file:
    json.dump(trainer.state.log_history, file, indent=4)